# Library Importation 

In [33]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
from datetime import datetime
import pytz
import tkinter as tk
from tkinter import messagebox


# Black-Scholes Model

In [34]:
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

def vega(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma* np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

 

# Newton-Raphson Method

In [35]:
def implied_volatility_newton(S, K, T, r, C, tol=1e-5, max_iter=100):
    sigma = 0.2 # Inital guess for volatility
    for _ in range(max_iter):
        price = black_scholes_call(S, K, T, r, sigma)
        vega_val = vega(S, K, T, r, sigma)
        diff = C - price
        if abs(diff) < tol:
            return sigma
        sigma = sigma + diff / vega_val
    return sigma
                          
    

# Bisection Method

In [36]:
def implied_volatility_bisection(S, K, T, r, C, tol=1e-5):
    low, high = 0.001, 1.0 # Assuming volatility within this range
    while high - low > tol:
        mid = (low + high) / 2
        price = black_scholes_call(S, K, T, r, mid)
        if price > C:
            high = mid
        else:
            low = mid
        return (low + high) / 2
    


# Example usage
S = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to expiration in years
r = 0.05 # Risk-free rate
C = 10   # Market price of the call option

print("Implied Volatility (Newton-Raphson):", implied_volatility_newton(S, K, T, r, C))
print("Implied Volatility (Bisection):", implied_volatility_bisection(S, K, T, r, C))
 

Implied Volatility (Newton-Raphson): 0.187971649526008
Implied Volatility (Bisection): 0.25075


In [37]:

# Assuming the functions above are defined here or imported

def calculate_iv():
    try:
        S = float(entry_S.get())
        K = float(entry_K.get())
        T = float(entry_T.get())
        r = float(entry_r.get())
        C = float(entry_C.get())
        
        iv_nr = implied_volatility_newton(S, K, T, r, C)
        iv_bis = implied_volatility_bisection(S, K, T, r, C)
        
        result.set(f"Newton-Raphson: {iv_nr:.4f}, Bisection: {iv_bis:.4f}")
    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid numbers for all fields.")

# GUI setup
root = tk.Tk()
root.title("Implied Volatility Calculator")

tk.Label(root, text="Stock Price (S)").grid(row=0)
entry_S = tk.Entry(root)
entry_S.grid(row=0, column=1)

tk.Label(root, text="Strike Price (K)").grid(row=1)
entry_K = tk.Entry(root)
entry_K.grid(row=1, column=1)

tk.Label(root, text="Time to Expiry (T)").grid(row=2)
entry_T = tk.Entry(root)
entry_T.grid(row=2, column=1)

tk.Label(root, text="Risk-Free Rate (r)").grid(row=3)
entry_r = tk.Entry(root)
entry_r.grid(row=3, column=1)

tk.Label(root, text="Option Price (C)").grid(row=4)
entry_C = tk.Entry(root)
entry_C.grid(row=4, column=1)

tk.Button(root, text="Calculate IV", command=calculate_iv).grid(row=5, column=0, columnspan=2)
result = tk.StringVar()
tk.Label(root, textvariable=result).grid(row=6, column=0, columnspan=2)

root.mainloop()